In [ ]:
#导入库
import random
import time
import os
import re
import requests
from lxml import etree
import pandas as pd 
import time
import re
from urllib.parse import urlencode

def get_mdlink(url,cookie):
    headers = {
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36", 
        "cookie": cookie,
        "pragma": "no-cache",
        "sec-fetch-dest": "document",
        "sec-fetch-mode": "navigate",
        "sec-fetch-site": "same-origin",
        "sec-fetch-user": "?1",
        "upgrade-insecure-requests": "1",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36"
    }
    
    r=requests.get(url,headers=headers,verify=False)
    html = r.text
    mdlink = "https:" + re.search("(.*)url=\'(.*)\',isg=",html).group(2)
    md_args = {
        "callback": "setMdskip",
        "timestamp": round(time.time() * 1000),
        #从cookie中拿
        "isg": re.search('(^|;) ?l=([^;]*)(;|$)',cookie).group(2),
        "isg2": re.search('(^|;) ?isg=([^;]*)(;|$)',cookie).group(2)
    }

    try:
        pre_arg = re.search("[?&^](ip|campaignId|key|abt|cat_id|q|u_channel|areaId|sdShopId)=([^&]+)",url).group(1)
        post_arg = re.search("[?&^](ip|campaignId|key|abt|cat_id|q|u_channel|areaId|sdShopId)=([^&]+)",url).group(2)
        md_args[pre_arg] = post_arg
    except:
        md_args
    
    mdlink = mdlink + "&" + urlencode(md_args)
    #print(mdlink)
    return mdlink

def get_info(url,yushou_cookie):

    headers = {
        "cookie": yushou_cookie,
        #headers 中的 referer 必不可少, 少了无妨访问
        "referer": "https://detail.tmall.com/",
        "sec-ch-ua":"""" Not A;Brand";v="99", "Chromium";v="90", "Google Chrome";v="90" """,
        "sec-ch-ua-mobile": "?0",
        "sec-fetch-dest": "script",
        "sec-fetch-mode": "no-cors",
        "sec-fetch-site": "cross-site",
        "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
    }
    
    r=requests.get(url,headers=headers,verify=False)
    return r.text


#导出文档
def download_csv(data):
    # 获取采集时间
    open_time = time.localtime(time.time())
    open_time = time.strftime("%Y-%m-%d", open_time) 
    
    #获取文件路径
    curt_path = os.path.abspath("__file__")
    filname = os.path.dirname(curt_path) + "/" + "预定件数_件单价" + "_" + open_time + ".csv"
    if os.path.exists(filname):
        os.remove(filname)
        
    #导出
    head=['id','预定件数', '件单价']
    details=pd.DataFrame(columns=head,data=data)
    print(details.info())
    #解决中文乱码
    details.to_csv(filname,encoding = "utf_8_sig")
    print(filname)


In [ ]:
url = "https://detail.tmall.com/item.htm?spm=a21ag.12100459.sycm-cc-macroscopic-monitor-item-rank-table.8.27cb50a5QO7v5u&id=601556634688"

In [ ]:
df = pd.read_excel("预售监控链接分工_0526.xlsx")

In [ ]:
df.head()

In [ ]:
links = df["预售商品链接"].tolist()
len(links)

In [ ]:

#-------------------执行部分-----------------------
#df = pd.read_excel("预售id.xlsx")
#id_list = df["ID"].tolist()
yushou_data = []
error_link = []
count = 0
#print("一共多少个id: ", len(id_list))
links = df["预售商品链接"].tolist()

#积累cookies池
cookies = []
yushou_cookies =  []

print("总共需取爬取的预售链接: ",len(links))
print("-"*50)

for link in links:
    count += 1
    time.sleep(random.uniform(0.5,1.5))
    
    try:
        print("爬取第{}个预售链接".format(str(count)))
        md_link = get_mdlink(link,random.choice(cookies))
        doc = get_info(md_link,random.choice(yushou_cookies))

        yushou_order = int(re.search('(.*)groupUC(.*)maxAmount',doc).group(2).split('"')[2])
        ##预售价有问题
        yushou_price = float(re.search('(.*)price(.*)promType',doc).group(2).split('"')[2])

        sku_id = re.search("[?&^](id)=([^&]+)",link).group(2)

        print((sku_id,yushou_order,yushou_price))
        yushou_data.append((sku_id,yushou_order,yushou_price))
        print("-"*50)

    except Exception as e:
        print("*"*25)
        print("第{}个预售链接爬取未成功".format(str(count)),e)
        print(link)
        print("*"*25)
        error_link.append(link)
        

print(yushou_data)
#print("全部预售链接爬取完毕, 开始愉快的统计叭!!!")
#download_csv(yushou_data)
